# <center> Data Loading </center> 

[API Госдумы](http://api.duma.gov.ru/pages/dokumentatsiya/primeri-zaprosov-k-api-ais-zakonoproekt)

In [1]:
import pandas as pd
import numpy as np

import json
import requests

from tqdm import tqdm_notebook
from pandas.io.json import json_normalize

import sys
sys.path.append('../')
from src.TranscriptsLoader import TranscriptsLoader

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Процедура получения API ключа описана [здесь](http://api.duma.gov.ru/key-request)

In [2]:
with open("../src/api_token.txt", 'r') as f:
    API_TOKEN = f.read()

In [3]:
requst_url = f"http://api.duma.gov.ru/api/{API_TOKEN}/deputies.json?position=Депутат%20ГД"
response = requests.get(requst_url, headers={'REFERER':"https://github.com/DmitrySerg"})
response_payload = json.loads(response.content)
response

<Response [200]>

In [4]:
deputies = json_normalize(response_payload, ['factions'], ['id', 'name', 'position'], record_prefix='_')
deputies.columns = [
    'party_id', 'party_name', 
    'start_date', 'end_date', 
    'deputy_id', 'deputy_name', 
    'deputy_position'
]

In [5]:
deputies.head()

,party_id,party_name,start_date,end_date,deputy_id,deputy_name,deputy_position
0,72100004,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",2011-12-04,2016-10-04,99111772,Абалаков Александр Николаевич,Депутат ГД
1,72100024,"Фракция Всероссийской политической партии ""ЕДИ...",2013-03-06,2016-10-04,99112284,Абасов Мамед Магарамович,Депутат ГД
2,72100020,"Депутатская группа ""Российские регионы""",1996-01-16,1997-11-12,99100491,Абдулатипов Рамазан Гаджимурадович,Депутат ГД
3,72100024,"Фракция Всероссийской политической партии ""ЕДИ...",2011-12-04,2013-01-27,99100491,Абдулатипов Рамазан Гаджимурадович,Депутат ГД
4,72100001,Фракция Аграрной партии России,1994-01-12,1994-07-23,99100001,Абдулбасиров Магомедтагир Меджидович,Депутат ГД


In [6]:
deputies.to_csv("../data/processed/deputies.csv")

In [7]:
deputies.shape

(4781, 7)

In [8]:
len(deputies['deputy_name'].unique())

2021

Тестовый прогон

In [9]:
transcripts_loader = TranscriptsLoader(
    deputy_id=99112284, 
    date_from='1994-01-01', 
    date_to='2021-03-31',
    api_token=API_TOKEN
)
transcripts_data = transcripts_loader.get_transcripts_data()

In [10]:
transcripts_data

,start_line,end_line,type,lines,votes,deputy_name,meeting_date,deputy_id
0,4904,4933,65100003,"[АБАСОВ М. М., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Абасов Мамед Магарамович,2016-04-15 00:00:00,99112284
1,4972,4988,,"[АБАСОВ М. М. Спасибо за вопрос. Я думаю, что ...",[],Абасов Мамед Магарамович,2016-04-15 00:00:00,99112284
2,6129,6133,,"[АБАСОВ М. М., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Абасов Мамед Магарамович,2016-03-16 00:00:00,99112284


Для каждого депутата получаем все записи выступлений

In [12]:
unique_deputy_ids = deputies['deputy_id'].unique().astype(int)

data = []
for deputy_id in tqdm_notebook(unique_deputy_ids, desc='Deputy'):
    transcripts_loader = TranscriptsLoader(
        deputy_id=deputy_id,
        date_from='1994-01-01', 
        date_to='2021-03-31',
        api_token=API_TOKEN
    )
    transcripts_data = transcripts_loader.get_transcripts_data()
    data.append(transcripts_data)
    
data_df = pd.concat(data, ignore_index=True)
data_df.to_pickle('../data/raw/transcripts_1994_2021.pkl')